In [12]:
import pandas as pd 
from pathlib import Path

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pypots.imputation import SAITS


2025-11-14 10:31:35 [WARNING]: ‼️ PyPOTS Ecosystem configuration file does not exist.
2025-11-14 10:31:35 [INFO]: Wrote new configs to config.ini successfully.
2025-11-14 10:31:35 [INFO]: 💫 Initialized PyPOTS Ecosystem configuration file C:\Users\marqu\.pypots\config.ini successfully.
c:\Users\marqu\Documents\DataChallengeEnedis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



### Fonctions utiles

In [13]:
def scale_df_log(df_log: pd.DataFrame, scaler: StandardScaler) -> pd.DataFrame:
    arr = df_log.to_numpy().astype(float)
    mask = ~np.isnan(arr)
    arr_scaled = np.full_like(arr, np.nan, dtype=float)
    arr_scaled[mask] = scaler.transform(arr[mask].reshape(-1, 1))[:, 0]
    return pd.DataFrame(arr_scaled, index=df_log.index, columns=df_log.columns)


In [17]:
def make_pypots_set_from_full(df_full: pd.DataFrame,
                              missing_rate: float,
                              rng: np.random.Generator):
    """
    df_full : (T, n_series) sans NaN (normalement)
    Retourne un dict {"X": array(N, T, 1)} avec NaN ajoutés artificiellement.
    N = n_series
    """
    arr = df_full.to_numpy().T  # (n_series, T)
    N, T_ = arr.shape
    X = arr[..., None].copy()   # (N, T, 1)

    # masque de manquants artificiels
    # (si tu veux faire plus sophistiqué, tu peux créer des segments au lieu de Bernoulli simple)
    miss_mask = rng.random(size=(N, T_)) < missing_rate
    X[miss_mask, 0] = np.nan  # 0 = seul feature

    return {"X": X}

In [22]:
def make_train_val_sets(df_full, missing_rate=0.12, val_ratio=0.1, rng=None):
    """
    df_full : dataframe (T, N_series) FULL (sans trou)
    missing_rate : proportion de NaN artificiels à injecter
    """

    if rng is None:
        rng = np.random.default_rng(42)

    # split colonnes (les séries)
    cols = df_full.columns.to_list()
    N = len(cols)
    N_val = int(N * val_ratio)
    val_cols = cols[:N_val]
    train_cols = cols[N_val:]

    train_df = df_full[train_cols]  # (T, N_train)
    val_df   = df_full[val_cols]    # (T, N_val)

    # TRAIN : on masque artificiellement, mais on ne fournit pas X_ori
    train_arr = train_df.to_numpy().T[:, :, None]    # (N_train, T, 1)
    mask_train = rng.random(train_arr.shape[:2]) < missing_rate   # (N_train, T)
    X_train = train_arr.copy()
    X_train[mask_train, 0] = np.nan

    train_set = {"X": X_train}

    # VAL : on masque artificiellement, ET on fournit X_ori original
    val_arr = val_df.to_numpy().T[:, :, None]        # (N_val, T, 1)
    mask_val = rng.random(val_arr.shape[:2]) < missing_rate
    X_val = val_arr.copy()
    X_val[mask_val, 0] = np.nan

    val_set = {
        "X": X_val,
        "X_ori": val_arr   # PyPOTS exige X_ori pour la validation
    }

    return train_set, val_set


### Importation des données d'entrainement et de test

In [3]:
BASE = Path("../data/processed")
CLUSTERS_PATH = BASE.parent / "clusters" / "cluster_labels_k4.csv"
X_tr = pd.read_csv(BASE / "X_train.csv", index_col=0,parse_dates=True)
X_test = pd.read_csv(BASE / "X_test.csv", index_col=0,parse_dates=True)
Y_tr = pd.read_csv(BASE / "y_train.csv", index_col=0,parse_dates=True)
clusters_labels = pd.read_csv(CLUSTERS_PATH, index_col=0)

In [4]:
holed_cols = [c for c in X_tr.columns if c.startswith("holed")]
clean_cols = [c for c in X_tr.columns if c not in holed_cols]

In [5]:
assert list(holed_cols) == list(Y_tr.columns)

### Normalisation des données avec StandardScaler

In [7]:
vals_X = X_tr.values.ravel()
vals_Y = Y_tr.values.ravel()
all_vals = np.concatenate([vals_X, vals_Y])

In [11]:
mask_obs = ~np.isnan(all_vals)
scaler = StandardScaler()
scaler.fit(all_vals[mask_obs].reshape(-1, 1))

,copy,True
,with_mean,True
,with_std,True


In [14]:
X_tr_scaled = scale_df_log(X_tr, scaler)
Y_tr_scaled = scale_df_log(Y_tr, scaler)
X_test_scaled = scale_df_log(X_test, scaler)

### Préparation des données l'entraînement et l'évaluation

In [15]:
# on remplace les 1 000 holed_ par leurs versions complètes Y_tr
all_train_full_scaled = pd.concat(
    [X_tr_scaled[clean_cols], Y_tr_scaled], axis=1
)  # shape (T, 21000)

T = all_train_full_scaled.shape[0]
n_series = all_train_full_scaled.shape[1]
print("T =", T, "n_series =", n_series)


T = 1057 n_series = 20999


In [16]:
train_cols, val_cols = train_test_split(
    all_train_full_scaled.columns, test_size=0.1, random_state=42
)

train_df = all_train_full_scaled[train_cols]
val_df = all_train_full_scaled[val_cols]


In [23]:
train_set, val_set = make_train_val_sets(
    all_train_full_scaled,
    missing_rate=0.12,
    val_ratio=0.1,
    rng=np.random.default_rng(42)
)

print(train_set["X"].shape)
print(val_set["X"].shape, val_set["X_ori"].shape)



(18900, 1057, 1)
(2099, 1057, 1) (2099, 1057, 1)


In [ ]:
n_steps = train_set["X"].shape[1]
n_features = train_set["X"].shape[2]

saits = SAITS(
    n_steps=n_steps,
    n_features=n_features,
    d_model=64,
    d_k=32,
    d_v=32,
    n_heads=4,
    n_layers=2,
    d_ffn=128,
    dropout=0.1,
    batch_size=64,
    epochs=50,
    patience=5,
    device="cpu"
)

saits.fit(train_set, val_set)


2025-11-14 10:41:52 [INFO]: Using the given device: cpu
2025-11-14 10:41:52 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-11-14 10:41:52 [INFO]: Using customized MAE as the training loss function.
2025-11-14 10:41:52 [INFO]: Using customized MSE as the validation metric function.
2025-11-14 10:41:52 [WARNING]: ‼️ d_model must = n_heads * d_k, it should be divisible by n_heads and the result should be equal to d_k, but got d_model=64, n_heads=4, d_k=32
2025-11-14 10:41:52 [WARNING]: ⚠️ d_model is reset to 128 = n_heads (4) * d_k (32)
2025-11-14 10:41:52 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 398,375
2025-11-14 11:34:55 [INFO]: Epoch 001 - training loss (MAE): 0.4513, validation MSE: 0.1491
2025-11-14 12:37:56 [INFO]: Epoch 002 - training loss (MAE): 0.3207, validation MSE: 0.1087
